Endpoint set to: http://localhost:5820/MyDB/update
HTTP authentication: method=basic, user=spencer, passwd set
Request format: ANY
Display: table

In [45]:
%endpoint http://localhost:5820/MyDB/update
%auth basic spencer 12345
%format any
%display table

PREFIX fhir: <http://hl7.org/fhir/>

DELETE WHERE { fhir:USCorePatient ?p ?o };
INSERT DATA {
    fhir:USCorePatient a owl:Class ;
        rdfs:subClassOf fhir:Patient ;
        rdfs:subClassOf [ a                  owl:Restriction ;
                          owl:onProperty     fhir:Patient.name ;
                          owl:maxCardinality 1
                        ] .
};

DELETE WHERE { fhir:NHSPatient ?p ?o };
INSERT DATA {
    fhir:NHSPatient a owl:Class ;
        rdfs:subClassOf fhir:Patient ;
        rdfs:subClassOf [ a                  owl:Restriction ;
                          owl:onProperty     fhir:Patient.name ;
                          owl:cardinality 1
                        ] .
};

DELETE WHERE { fhir:PatientProfileWithNameCardinalityOne ?p ?o };
INSERT DATA {
    fhir:PatientProfileWithNameCardinalityOne a owl:Class ;
        rdfs:subClassOf fhir:Patient ;
        rdfs:subClassOf [ a                  owl:Restriction ;
                          owl:onProperty     fhir:Patient.name ;
                          owl:cardinality 1
                        ] .
};

Endpoint set to: http://localhost:5820/MyDB/update
HTTP authentication: method=basic, user=spencer, passwd set
Request format: ANY
Display: table

In [49]:
%endpoint http://localhost:5820/MyDB/query
%auth basic spencer 12345
%format any
%display table

PREFIX fhir: <http://hl7.org/fhir/>

# Get all instances of profile pairs (cls1 cls2) where the profiles
# restrict cardinality in a different way.
SELECT DISTINCT ?cls1 ?cls2 ?property ?restriction_type1 ?v1 ?restriction_type2 ?v2
WHERE {
    ?cls1 rdfs:subClassOf fhir:Patient .
    ?cls2 rdfs:subClassOf fhir:Patient .
    fhir:Patient rdfs:subClassOf [ a              owl:Restriction ;
                                   owl:onProperty ?property ;
                                 ] .
    ?cls1 rdfs:subClassOf  [ a                  owl:Restriction ;
                             owl:onProperty     ?property ;
                             ?restriction_type1 ?v1
                           ] .
    ?cls2 rdfs:subClassOf  [ a                  owl:Restriction ;
                             owl:onProperty     ?property ;
                             ?restriction_type2 ?v2
                           ] .

    FILTER (?cls1 != ?cls2)
    FILTER (?restriction_type1 IN (owl:cardinality, owl:maxCardinality, owl:minCardinality ) ) .
    FILTER (?restriction_type2 IN (owl:cardinality, owl:maxCardinality, owl:minCardinality ) ) .
    FILTER (?restriction_type1 != ?restriction_type2 || ?v1 != ?v2) .
}

Endpoint set to: http://localhost:5820/MyDB/query
HTTP authentication: method=basic, user=spencer, passwd set
Request format: ANY
Display: table

cls1,cls2,property,restriction_type1,v1,restriction_type2,v2


In [46]:
%endpoint http://localhost:5820/MyDB/query
%auth basic spencer 12345
%format any
%display table

PREFIX fhir: <http://hl7.org/fhir/>

SELECT ?cls WHERE { ?cls rdfs:subClassOf fhir:Patient }

Endpoint set to: http://localhost:5820/MyDB/query
HTTP authentication: method=basic, user=spencer, passwd set
Request format: ANY
Display: table

cls
http://hl7.org/fhir/USCorePatient
http://hl7.org/fhir/NHSPatient
http://hl7.org/fhir/PatientProfileWithNameCardinalityOne
